In [0]:
dbutils.widgets.text("proc_date", "", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_mdl_winlife_asm_dr_tracking
AS

WITH main AS (
  SELECT * EXCEPT(proc_date)
  FROM {catalog_name}.udp_wcm_silver_dataportal.mdl_winlife_asm_dr_tracking
  WHERE upload_user IN ('hangbtl@mml.masangroup.com', 'haontt@winmart.masangroup.com', 'thimpt@winmart.masangroup.com')
  QUALIFY DENSE_RANK() OVER (PARTITION BY upload_user ORDER BY upload_time DESC) = 1
)

SELECT *
FROM main
QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY upload_time DESC) = 1
;
""")